In [ ]:
# Аналитическая задача -- составить портрет клиента, склонного откликнуться на предложение о новой карте.
#Шаг 1. Загрузка данных;
#Шаг 2. Первичная обработка данных (при необходимости):
#- скорректировать заголовки;
#- скорректировать типы признаков;
#- проверить наличие дублирующихся записей;
#- проверить наличие аномальных значений;
#- восстановить пропущенные значения;
#Шаг 3. Исследовательский анализ данных
#- в разрезе значений целевого признака (`response` -- Отклик на предложение новой карты	) исследовать распределения признаков;
#- исследовать возможные зависимости целевого признака от объясняющих признаков;
#- в разрезе целевого признака составить портреты клиентов платежной системы;
#Шаг 4. Составить и проверить гипотезы о наличие/отсутствии различий по признакам портрета клиента.
#Шаг 5. Построить классификационные модели (дополнительное задание).

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import mannwhitneyu
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
import shap
import xgboost

df = pd.read_csv('/content/drive/MyDrive/Аналитик Данных/Итоговая работа/project_7/vrk_response_bank.csv', sep=';')
#df = pd.read_csv('vrk_response_bank.csv', sep=';')
df.shape

# приводим заголовки к нижнему регистру
df.columns = df.columns.str.lower()
# переименуем признаки
df = df.rename(columns={
        'mortgage':'Ипотечный_кредит' ,
        'life_ins':'Страхование_жизни',
        'cre_card':'Кредитная_карта',
        'deb_card':'Дебетовая_карта',
        'mob_bank':'Мобильный_банк',
        'curr_acc':'Текущий_счет',
        'internet':'Интернет_доступ',
        'perloan':'Индивидуальный_заем',
        'savings':'Наличие_сбережений',
        'atm_user':'Пользование_банкоматом_последнюю_неделю',
        'markpl':'Пользование_маркетплейса_последний_месяц',
        'age':'Возраст',
        'cus_leng':'Давность_клиентской_истории'
        #'response':'отклик'
        })

# приводим заголовки к нижнему регистру
df.columns = df.columns.str.lower()

# Учитывая что все признаки числовые произведем условное разделение признаков по количеству значений в признаке
numerical = [col for col in df.columns if len(df[col].value_counts()) > 5]
categorical = [col for col in df.columns if len(df[col].value_counts()) <= 5]
print(numerical)
print(categorical)

# Удалим дубликаты, просмотрим изменения целевой переменной
print('до удаления дублей', df.shape)
print(df['response'].value_counts())
df.drop_duplicates(inplace=True)
print('после удаления дублей', df.shape)
print(df['response'].value_counts())

# Расчет квантилей
def calculate_iqr_boundaries(series):
  q25 = series.quantile(0.25)
  q75 = series.quantile(0.75)
  iqr = q75 - q25

  boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)
  return boundaries

# Заменим значения выбросов граничными значениями квантильного размаха
numeric = df.drop(['response'], axis=1).columns

for i in numerical:
    bounds = calculate_iqr_boundaries(df[i])
    out_l = sum(df[i] < bounds[0])
    out_r = sum(df[i] > bounds[1])
    # Заменяем выбросы на границы квантильного размаха
    df.loc[df[i] < bounds[0], i] = bounds[0]
    df.loc[df[i] > bounds[1], i] = bounds[1]

    # Произведем повторный расчет выбросов
    bounds = calculate_iqr_boundaries(df[i])
    out_l = sum(df[i] < bounds[0])
    out_r = sum(df[i] > bounds[1])
    # Заменяем выбросы на границы квантильного размаха
    df.loc[df[i] < bounds[0], i] = bounds[0]
    df.loc[df[i] > bounds[1], i] = bounds[1]

    print('-------', i, '-------')
    print('Размах', bounds)
    print('Количество выбросов', out_l, out_r)
    #plt.boxplot(df[i])
    #plt.show()
    print('=======+++++++=======')

print(df.isnull().sum())

## Исследование распределения признаков в разрезе значений целевого признака (`response` -- Отклик на предложение новой карты)

# Посмотрим оценки выборочных средних и медиан для каждого признака в разрезе целевого признака
cols = numerical

grouped = df.groupby('response')

for col in cols:
    mean_resp = grouped[col].mean()
    median_resp = grouped[col].median()
    print(f'For {col}:')
    print('Среднее:', mean_resp)
    print('Медиана:', median_resp)

# Создаем графики распределения для каждого числового признака по значениям response
for feature in numerical:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='response', y=feature, data=df)
    plt.title(f'Распределение {feature} по значениям response')
    plt.xlabel('response')
    plt.ylabel(feature)
    plt.grid()
    plt.show()

# гистограммы и функции плотности для каждого признака в разрезе целевого признака
plt.figure(figsize=(15, 5))

for i, col in enumerate(numerical, 1):
    plt.subplot(1, 3, i)
    for response, data in df.groupby('response'):
        data[col].plot(kind='kde', label=f'response {response}', alpha=0.5)
    plt.title(col)
    plt.legend()

plt.show()

column = df.drop(['response'], axis=1).columns
# Построим графики для всех признаков
fig, ax = plt.subplots(13, 2, figsize=(21, 45))  # Увеличил высоту

for i, y in enumerate(column):
    plt.subplot(13, 2, 2*i+1)
    mean_0 = df[df.response == 0][y].mean()
    median_0 = df[df.response == 0][y].median()

    ax = sns.histplot(data=df[df.response == 0],
                      x=y,
                      color='green',
                      label='принявшие')
    ax = sns.histplot(data=df[df.response == 1],
                      x=y,
                      color='red',
                      label='отказавшиеся')
    ax.axvline(mean_0, color='green', label='среднее принявших')
    ax.axvline(median_0, color='green', label='медиана принявших')
    plt.title(y, fontsize=12)  # Добавил заголовок для графика
    plt.legend()

    plt.subplot(13, 2, 2*i+2)
    mean_1 = df[df.response == 1][y].mean()
    median_1 = df[df.response == 1][y].median()

    ax = sns.kdeplot(data=df[df.response == 0],
                      x=y,
                      color='green',
                      label='принявшие')
    ax = sns.kdeplot(data=df[df.response == 1],
                      x=y,
                      color='red',
                      label='отказавшиеся')

    ax.axvline(mean_1, color='red', label='среднее отказавшихся')
    ax.axvline(median_1, color='red', label='медиана отказавшихся')

    plt.title(y, fontsize=12)  # Добавил заголовок для графика
    plt.legend()

plt.tight_layout()  # Убирает перекрытие подзаголовков и графиков
plt.show()

## Исследования возможных зависимостей целевого признака от объясняющих признаков
# Построим матрицу корреляций
corr = df.corr()
sns.heatmap(corr, annot=True)

# Анализ распределения числовых признаков
sns.pairplot(df, hue='response')

## Составление портретов клиентов платежной системы в разрезе целевого признака
# Портрет клиентов с response = 0
df_response_0 = df[df['response'] == 0]
mean_values_response_0 = df_response_0.mean()
# Портрет клиентов с response = 1
df_response_1 = df[df['response'] == 1]
mean_values_response_1 = df_response_1.mean()
# Сравнение портретов
print("Портрет клиентов с response = 0:")
print(mean_values_response_0)
print("Портрет клиентов с response = 1:")
print(mean_values_response_1)

# 4. Составление и проверка гипотезы о наличие/отсутствии различий по признакам портрета клиента
# проверка нормальности распределений признаков
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
for feature in numeric_features:
    stat, p = stats.shapiro(df[feature])
    alpha = 0.05
    if p > alpha:
        print(f'Признак: {feature} имеет нормальное распределение (p-value={p})')
    else:
        print(f'Признак: {feature} не имеет нормального распределения (p-value={p})')

# Создадим два датафрейма - df_response_0 и df_response_1, содержащие строки с response=0 и response=1 соответственно.
df_response_0 = df[df['response'] == 0]
df_response_1 = df[df['response'] == 1]
# посчитаем средние значения признаков для каждой из групп.
mean_values_response_0 = df_response_0.mean()
mean_values_response_1 = df_response_1.mean()

# проверим гипотезу о равенстве средних значений признаков двух групп.
# применяем mannwhitneyu дле признаков с ненормальным распределением
from scipy.stats import ttest_ind
alpha = 0.05  # задаем уровень значимости
for feature in df.columns:
    t_stat, p_value = mannwhitneyu(df_response_0[feature], df_response_1[feature])
    if p_value < alpha:
        print(f"Отвергаем нулевую гипотезу для признака {feature}: средние значения различаются")
    else:
        print(f"Не отвергаем нулевую гипотезу для признака {feature}: средние значения не различаются")

# Разделение данных на признаки и целевую переменную
X = df.drop('response', axis=1)
y = df['response']
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Инициализация и обучение модели
model = RandomForestClassifier(class_weight='balanced')
model.fit(X_train, y_train)
# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test)
# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 score: {f1}')



# Кодирование категориальных признаков трейн и тест (тест на обученном препроцессоре)
def encode_category_feature(X, df_predict, category_feature_list):
    encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
    # Применение OneHotEncoder
    encoded_train = encoder.fit_transform(X[category_feature_list])
    # кодируем предикт
    encoded_predict = encoder.transform(df_predict[category_feature_list])
    # Преобразование результата обратно в DataFrame
    encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(category_feature_list))
    encoded_predict_df = pd.DataFrame(encoded_predict, columns=encoder.get_feature_names_out(category_feature_list))
    # Сброс индексов
    X.reset_index(drop=True, inplace=True)
    df_predict.reset_index(drop=True, inplace=True)

    encoded_train_df.reset_index(drop=True, inplace=True)
    encoded_predict_df.reset_index(drop=True, inplace=True)
    # Объединение с сбросом индексов
    X_encoded = pd.concat([X, encoded_train_df], axis=1)
    predict_encoded = pd.concat([df_predict, encoded_predict_df], axis=1)
    # Удаление оригинального столбца
    X_encoded = X_encoded.drop(columns=category_feature_list)
    predict_encoded = predict_encoded.drop(columns=category_feature_list)

    return X_encoded, predict_encoded



def standard(X, df_predict, numeric):
    # Применяем StandardScaler к X, numeric
    st_scaler = StandardScaler()
    X[numeric] = st_scaler.fit_transform(X[numeric])
    # Применяем StandardScaler к predict
    df_predict[numeric] = st_scaler.transform(df_predict[numeric])

    return X, df_predict

category = df.drop(['response', 'возраст'], axis=1).columns
X_train_enc, X_test_enc = encode_category_feature(X_train, X_test, category)
X_train_norm, X_test_norm = standard(X_train_enc, X_test_enc, numerical)
# Инициализация и обучение модели
model = RandomForestClassifier(class_weight='balanced')
model.fit(X_train_norm, y_train)
# Прогнозирование на тестовой выборке
y_pred = model.predict(X_test_norm)
# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 score: {f1}')

X_train, X_test = X_train_norm, X_test_norm
# Логистическая регрессия
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_probs = lr_model.predict_proba(X_test)[:,1]
roc_auc_lr = roc_auc_score(y_test, lr_probs)
f1_lr = f1_score(y_test, lr_probs>0.5)
confusion_lr = confusion_matrix(y_test, lr_probs>0.5)
# Случайный лес
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_probs = rf_model.predict_proba(X_test)[:,1]
roc_auc_rf = roc_auc_score(y_test, rf_probs)
f1_rf = f1_score(y_test, rf_probs>0.5)
confusion_rf = confusion_matrix(y_test, rf_probs>0.5)
# Метод опорных векторов
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_probs = svm_model.predict_proba(X_test)[:,1]
roc_auc_svm = roc_auc_score(y_test, svm_probs)
f1_svm = f1_score(y_test, svm_probs>0.5)
confusion_svm = confusion_matrix(y_test, svm_probs>0.5)
print("Логистическая регрессия:")
print("ROC AUC:", roc_auc_lr)
print("F1 Score:", f1_lr)
print("Confusion Matrix:")
print(confusion_lr)
print("\nCлучайный лес:")
print("ROC AUC:", roc_auc_rf)
print("F1 Score:", f1_rf)
print("Confusion Matrix:")
print(confusion_rf)
print("\nМетод опорных векторов:")
print("ROC AUC:", roc_auc_svm)
print("F1 Score:", f1_svm)
print("Confusion Matrix:")
print(confusion_svm)

# Удаление признаков с низкой корреляцией
def high_corr(df):
    # сразу удалим id
    df.drop(['id'], axis=1, inplace=True, errors='ignore')
    min_corr = []
    for i in df.columns:
        j = df[i]
        corr = df['response'].corr(j)
        #print(i, corr)
        if -0.01 < corr and corr < 0.01:
            min_corr.append(i)

    print(min_corr)
    df.drop(min_corr, axis=1, inplace=True)

    return df



# Удаление признаков с нулевой значимостью
def important(df):
    # Определение данных
    X = df.drop(['response'], axis=1)
    y = df['response']
    # Обучение модели
    params = {
        'objective': 'binary:logistic',
        'max_depth': 3,
        'learning_rate': 0.1
        }
    model = xgboost.train(params, xgboost.DMatrix(X, y), num_boost_round=10)
    # Создание объекта Explainer и получение SHAP значений
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    # Преобразуем shap_values в DataFrame для удобной работы
    shap_df = pd.DataFrame(shap_values, columns=X.columns)
    # Создание переменной no_good для признаков с нулевой значимостью
    no_good = shap_df.mean().index[shap_df.mean() == 0].tolist()
    # Выводим список признаков с нулевой значимостью
    print("Признаки с нулевой значимостью:")
    print(no_good)
    df.drop(no_good, axis=1, inplace=True)

    return df

df = high_corr(df)
df = important(df)
# Разделение данных на признаки и целевую переменную
X = df.drop('response', axis=1)
y = df['response']
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Учитывая что все признаки числовые произведем условное разделение признаков по количеству значений в признаке
numerical = [col for col in X_train.columns if len(X_train[col].value_counts()) > 5]
categorical = [col for col in X_train.columns if len(X_train[col].value_counts()) <= 5]
print(numerical)
print(categorical)

# кодируем и нормализуем
X_train_enc, X_test_enc = encode_category_feature(X_train, X_test, categorical)
X_train_norm, X_test_norm = standard(X_train, X_test, numerical)

# Логистическая регрессия
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_probs = lr_model.predict_proba(X_test)[:,1]
roc_auc_lr = roc_auc_score(y_test, lr_probs)
f1_lr = f1_score(y_test, lr_probs>0.5)
confusion_lr = confusion_matrix(y_test, lr_probs>0.5)

# Случайный лес
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_probs = rf_model.predict_proba(X_test)[:,1]
roc_auc_rf = roc_auc_score(y_test, rf_probs)
f1_rf = f1_score(y_test, rf_probs>0.5)
confusion_rf = confusion_matrix(y_test, rf_probs>0.5)

# Метод опорных векторов
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_probs = svm_model.predict_proba(X_test)[:,1]
roc_auc_svm = roc_auc_score(y_test, svm_probs)
f1_svm = f1_score(y_test, svm_probs>0.5)
confusion_svm = confusion_matrix(y_test, svm_probs>0.5)

print("Логистическая регрессия:")
print("ROC AUC:", roc_auc_lr)
print("F1 Score:", f1_lr)
print("Confusion Matrix:")
print(confusion_lr)
print("\nCлучайный лес:")
print("ROC AUC:", roc_auc_rf)
print("F1 Score:", f1_rf)
print("Confusion Matrix:")
print(confusion_rf)
print("\nМетод опорных векторов:")
print("ROC AUC:", roc_auc_svm)
print("F1 Score:", f1_svm)
print("Confusion Matrix:")
print(confusion_svm)



Ваш код охватывает множество аспектов аналитической задачи по созданию профиля клиента, откликающегося на предложение новой карты. Рассмотрим его выполнение по шагам, приведенным в задаче, и дадим рекомендации по улучшению.

### Шаг 1. Загрузка данных

- **Реализация**: Произведена корректная загрузка данных.
- **Рекомендация**: Убедитесь, что путь к файлу доступен, особенно если вы планируете запускать код в разных средах.

### Шаг 2. Первичная обработка данных

- **Корректировка заголовков**: Заголовки приведены к нижнему регистру, что является хорошей практикой.
- **Типы признаков**: Убедитесь, что типы признаков установлены корректно, и при необходимости выставьте их.
- **Проверка дубликатов**: Удаление дубликатов выполнено.
- **Проверка аномальных значений**: Идея использования IQR для выбросов хороша, однако стоит ограничить замену на границы квантиля к первой итерации. Повторное применение этой логики может привести к нежелательным результатам.
- **Восстановление пропущенных значений**: Отсутствует. Рекомендуется добавить, например, замены медианой для числовых признаков или самым частым значением для категориальных.

### Шаг 3. Исследовательский анализ данных

- **Анализ распределений**: Визуализации проведены на высшем уровне. Корректная идея использования boxplot и KDE.
- **Портрет клиентов с `response=0` и `response=1`**: Построение портретов осуществлено через средние значения. Можно дополнить детализацией, например, через средние, медианы и стандартные отклонения.

### Шаг 4. Проверка гипотез

- **Проверка нормальности**: Подходящая реализация, однако сам метод Шапиро-Уилка может быть чувствителен к выборкам. Посмотрите также на тесты на нормальность по другим методам (например, Kolmogorov-Smirnov).
- **Проверка различий**: Использование теста Манна-Уитни правильно, так как данные, скорее всего, не имеют нормального распределения. Статистический анализ проведен.

### Шаг 5. Построение классификационных моделей

- **Создание и проверка моделей**: Модели логистической регрессии, случайного леса и метода опорных векторов построены. Результаты ROC AUC и F1 Score предоставлены.
- **Рекомендации**:
  - Рассмотрите пересечение методов кросс-валидации для более надежной оценки модели.
  - Внедрите другие алгоритмы, такие как градиентный бустинг (например, LightGBM) или более сложные ансамбли.
  - Проверьте влияние различных значений гиперпараметров, например, с использованием `GridSearchCV`.

### Общие рекомендации

1. **Визуализация важности признаков**: Посмотрите на важность признаков для выбранной модели (например, случайного леса), чтобы понять, какие из них вносят больше всего в предсказание.
2. **Обработка категориальных признаков**: Убедитесь, что все методы кодирования (например, OneHotEncoding) правильно применяются к обучающей и тестовой выборкам.
3. **Отладка после вычисления моделей**: В случае обнаружения низкой точности моделей, попробуйте выделить категориальные признаки, а затем провести дополнительные преобразования данных.
4. **Документирование каждого шага**: Обратитесь к документации, чтобы специфицировать дополнительные комментарии к коду и объяснить каждую часть процесса.
5. **Изучение различных метрик**: Рассмотрите использование других метрик оценки, таких как AUC-ROC и PR Curve, особенно в несбалансированных выборках.

### Заключение

Ваш код уже довольно хорош и охватывает большинство необходимых шагов для анализа данных и создания модели. Однако улучшения в плане проверки на пропущенные значения, отладка модели и визуализация важности признаков могут значительно повысить качество анализа и понимание ваших данных.